In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from brian2 import *  

C:\Users\odhra\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
mnist = keras.datasets.mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data() 

print(train_images.shape)

train_im = train_images.reshape(60000,784)

print(train_im.shape)

td = zeros((120000,784))


#maybe make this twice as big for a training and testing phase?

for x in range(0,60000):
    td[2*x]=train_im[x]
    

timed_data = td

print(timed_data.shape)

(60000, 28, 28)
(60000, 784)
(120000, 784)


In [3]:
start_scope()

taum = 10*ms
taupre = 20*ms
taupost = taupre
Ee = 0*mV
vt = -54*mV
vr = -60*mV
El = -74*mV
taue = 5*ms
F = 15*Hz
gmax = .01
dApre = .01
dApost = -dApre * taupre / taupost * 1.05
dApost *= gmax
dApre *= gmax



eqs_neurons = '''
dv/dt = (ge * (Ee-vr) + El - v) / taum : volt
dge/dt = -ge / taue : 1
'''


ta = TimedArray((timed_data)*Hz, 10*ms)

#exposure time is 20 ms.



Poisson_layer = PoissonGroup(784,rates = 'ta(t, i)')
layer_V1 = NeuronGroup(784, eqs_neurons, threshold='v>vt', reset='v = vr', refractory=2*ms, method='linear')
layer_V2 = NeuronGroup(784, eqs_neurons, threshold='v>vt', reset='v = vr', refractory=2*ms, method='linear')


#poisson to layer 1
P_to_V1 = Synapses(Poisson_layer, layer_V1,
             '''w : 1
                dApre/dt = -Apre / taupre : 1 (event-driven)
                dApost/dt = -Apost / taupost : 1 (event-driven)''',
             on_pre='''ge += w
                    Apre += dApre
                    w = clip(w + Apost, 0, gmax)''',
             on_post='''Apost += dApost
                     w = clip(w + Apre, 0, gmax)''',
             )
P_to_V1.connect()
P_to_V1.w = 'rand() * gmax'



#layer 1 to layer 2
V1_to_V2 = Synapses(layer_V1, layer_V2,
             '''w : 1
                dApre/dt = -Apre / taupre : 1 (event-driven)
                dApost/dt = -Apost / taupost : 1 (event-driven)''',
             on_pre='''ge += w
                    Apre += dApre
                    w = clip(w + Apost, 0, gmax)''',
             on_post='''Apost += dApost
                     w = clip(w + Apre, 0, gmax)''',
             )
V1_to_V2.connect(condition='i!=j', p=0.5)
V1_to_V2.w = 'rand() * gmax'



#spikemon_P_naieve = SpikeMonitor(Poisson_layer)
#spikemon_V1_naieve = SpikeMonitor(layer_V1)
#spikemon_V2_naieve = SpikeMonitor(layer_V2)

#don't monitor stuff until the second time round!




# Run the network for initial exposure to all examples, then run the network for training examples. with STP turned off 

In [ ]:
run(xxxx*ms)


store('exposed_net')

for test_number in range(1):
    restore('eposed_net')
    P_to_V1.plastic = False
    V1_to_V2.plastic = False
    spikemon_V2 = SpikeMonitor(layer_V2)
    run(...)
    


In [ ]:
trains = spikemon_V2.spike_trains()

trains_pre_array = np.array(list(trains.values()))*1000

spikematrix = zeros((784,800)) # check you have this bit the right way around! 


for x in range(0,784):
    results = list(map(int, trains_pre_array[x]))
    spikematrix[x,results] = 1


mylist = hsplit(spikematrix,20)#horizontally splits the spike matrix into matrices 20 along!

myarray = np.array(mylist)

print(myarray.shape)

(num_kernels, kernel_size = filter_length, padding = 'valid',
            input_shape = input_shape, dilation_rate = 3))

In [26]:
class_names = ['zero', 'one', 'two', 'three', 'four', 
               'five', 'six', 'seven', 'eight', 'nine']

model = keras.Sequential([
    keras.layers.Conv1D(128,2,strides=1,padding="valid",input_shape=(28,28)),
    keras.layers.Flatten(),
    keras.layers.Dense(30, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])


model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



In [27]:
model.fit(train_images, train_labels, epochs=5)



Epoch 1/5
60000/60000 [==============================] - 9s 144us/step - loss: 11.1691 - acc: 0.3060
Epoch 2/5
60000/60000 [==============================] - 8s 139us/step - loss: 9.1640 - acc: 0.4310
Epoch 3/5
60000/60000 [==============================] - 8s 135us/step - loss: 9.0226 - acc: 0.4400
Epoch 4/5
60000/60000 [==============================] - 8s 140us/step - loss: 8.7259 - acc: 0.4585
Epoch 5/5
60000/60000 [==============================] - 8s 135us/step - loss: 7.8675 - acc: 0.5116


In [135]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Test accuracy:', test_acc)


TypeError: data type not understood

In [ ]:
predictions = model.predict(test_images)

predictions[0]